# Fonction d'ordre supérieur

Les fonctions d'ordre supérieur dans Spark SQL sont des fonctions qui prennent d'autres fonctions comme arguments ou qui retournent des fonctions. Ces fonctions permettent d'effectuer des opérations complexes sur des données structurées telles que des tableaux ou des structures dans Spark SQL.

À partir de la version 2.4, Spark SQL a introduit plusieurs fonctions d'ordre supérieur pour travailler avec des données complexes. Elles ne sont disponibles avec l'API Python que depuis la [version 3.1 de Spark](https://issues.apache.org/jira/browse/SPARK-30681). Voici quelques exemples de fonctions d'ordre supérieur couramment utilisées dans Spark SQL :

* **transform** : Applique une fonction donnée à chaque élément d'un tableau et retourne un nouveau tableau avec les résultats.<br />Syntaxe : `transform(array, function)`.
* **filter** : Retourne un nouveau tableau contenant les éléments qui satisfont la condition spécifiée par la fonction donnée.<br />Syntaxe : `filter(array, function)`.
* **exists** : Vérifie si au moins un élément d'un tableau satisfait la condition spécifiée par la fonction donnée.<br />Syntaxe : `exists(array, function)`.
* **forall** : Vérifie si tous les éléments d'un tableau satisfont la condition spécifiée par la fonction donnée.<br />Syntaxe : `forall(array, function)`.
* **aggregate** : Agrège les éléments d'un tableau à l'aide d'une fonction d'agrégation et d'une valeur initiale.<br />Syntaxe : `aggregate(array, initial_value, merge_function[, finish_function])`.

## Préambule

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("Spark SQL - Fonction d'ordre supérieur") \
    .master("local[*]") \
    .config("spark.ui.showConsoleProgress", "True") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

# Access the JVM and import the required Java classes
jvm = spark.sparkContext._jvm
Level = jvm.org.apache.logging.log4j.Level
Configurator = jvm.org.apache.logging.log4j.core.config.Configurator

# Set the root level to OFF
Configurator.setRootLevel(Level.OFF)

## Chargement

Nous allons revenir sur les commandes clients dans une cafétéria.

In [ ]:
dataframe = spark.read \
    .option("header", "true") \
    .schema("id STRING, client STRING, timestamp TIMESTAMP, product STRING, price DOUBLE") \
    .csv("data/orders.csv")

orders = dataframe.withColumnRenamed("client", "clientId")

orders.createOrReplaceTempView("orders")

## Ensemble des prix par jour

Nous voulons actuellement récupérer pour chaque jour, la liste des prix des produits vendus. Pour cela, nous allons utiliser la  fonction `collect_list(col)`.

La fonction `collect_list()` est en quelque sorte l'inverse de la fonction `explode()` : au lieu de décomposer une colonne de type liste en ligne dans le dataframe, `collect_list()` ressemble des lignes du dataframe pour former une colonne de type liste.

ATTENTION !!! La quantité de données collectées dans chaque liste ne devrait pas dépasser ~50 000 éléments. Au-delà de cette valeur, vous risquez de dépasser la capacité mémoire de vos exécuteurs.

In [ ]:
pricesByDay = orders \
    .groupBy(to_date(col("timestamp")).alias("date")) \
    .agg(collect_list(col("price")).alias("prices"))

pricesByDay.show(10, True)

## Somme des prix par jour

Utilisez la fonction `aggregate(col, init, f)` pour calculer pour chaque jour le prix total de vente.
 * `col` est la colonne qui contient la liste à agréger.
 * `init` est la valeur initiale (ou la valeur à retourner si la liste dans `col` est vide). Cette valeur est de type colonne.
 * `f` est la fonction lambda d'agrégation. Elle a pour signature `lambda accumulator, value: new_value`. Le premier paramètre de la fonction correspond au résultat intermédiaire, sachant que le premier résultat intermédiaire correspond à `init`. Le second paramètre correspond à une valeur provenant de `col`.


In [ ]:
result = ???

result.show(10, False)
result.explain()

## Somme des prix hauts par jour

Nous voulons calculer pour chaque jour le prix total de vente pour les articles de 2 EUR ou plus.

Utilisez la fonction `filter(col, f)` pour retirer du calcul les prix de moins de 2 EUR.
 * `col` est la colonne qui contient la liste à agréger
 * `f` est la fonction de filtrage. Elle a [deux formes](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.filter.html#pyspark.sql.functions.filter). La plus commune permet de dériver un booléen à partir de chaque valeur provenant de `col` (Signature : `(x: Column) -> Column`). La seconde forme permet d'avoir un index en plus de la valeur x (`(x: Column, i: Column) -> Column` où i est un index commençant à 0).

In [ ]:
val result = ???

result.show(limit=10, True)
result.explain()

Refaites l'exercice, mais en utilisant cette fois une requête SQL.

Vous aurez besoin de :
 * `CAST(col AS type)` : converti le type d'une colonne.
 * `FILTER(col, f)` : filtre des valeurs dans `col` selon la fonction `f`, où `f` est une fonction de la forme `col -> condition_sur_col`.
 * `AGGREGATE(col, init, f)` : agrège les données d'une colonne selon `f`, où `f` est une fonction de la forme `(résultat, col) -> agrégation_de_résultat_et_col`.

In [ ]:
result = spark.sql("""
SELECT
  date,
  ??? AS total
FROM (
  SELECT
    to_date(timestamp) AS date,
    collect_list(price) AS price
  FROM orders
  GROUP BY date
)
""")

result.show(10, False)
result.explain()

## Prix moyen par jour

Pour chaque journée, nous voulons calculer le prix moyen. Malheureusement, Spark ne fournit pas de fonction qui permet de calculer une moyenne sur une liste :/

Néanmoins, sans avoir à recourir à une UDF, avec l'aide de la fonction `aggregate()`, nous pouvons calculer une telle moyenne. Pour cela, nous devons :
 1. D'un côté compter le nombre de prix et de l'autre calculer la somme des prix.
 2. Faire la division entre le somme des prix et le décompte pour avoir la moyenne.

Il y a deux façons de faire la première étape : soit utiliser 2 fois `aggregate()` pour compter et faire la somme en parallèle, soit utiliser `aggregate()` une seule fois avec une sous-structure qui stocke le décompte et la somme en même temps.

À vous de voir :)

Vous aurez potentiellement besoin de :
 * la fonction `struct(column_1, column_2, ...)` : permet de créer une sous structure.
 * la méthode `.cast(type)` sur une colonne : permet de convertir le type d'une colonne selon le [type passé en paramètre](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/data_types.html). 
 * la syntaxe `<column>("field_name")` sur une colonne : permet d'accéder au champ `"field_name"` d'une colonne, si celle-ci contient une sous-structure.

In [ ]:
from pyspark.sql.types import *

result = ???

result.show(10, True)
result.explain()